In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
path="/Users/karolinegriesbach/Library/CloudStorage/OneDrive-InnkeeprUG/PerformanceMeasurement/Baselineanalyse/campaign_base_analysis/dtw_lillydoo_result_report.csv"
df = pd.read_csv(path,sep=",")
df.head()

In [26]:
df["labels"]=None
df["labels"] = np.where(
    (df["has_incident"] == True) & (df["predicted_incident"] == True), "tp", df["labels"]
)
df["labels"] = np.where(
    (df["has_incident"] == False) & (df["predicted_incident"] == True), "fp", df["labels"] # annoying, because a lot of false alarms
)
df["labels"] = np.where(
    (df["has_incident"] == True) & (df["predicted_incident"] == False), "fn", df["labels"] # these cost us, becasue then we are missing an alarm
)
df["labels"] = np.where(
    (df["has_incident"] == False) & (df["predicted_incident"] == False), "tn", df["labels"]
)
df["labels"].value_counts()

In [10]:
perf_by_treatment = df.groupby("treatment")["labels"].value_counts().reset_index()
perf_by_treatment = pd.pivot(perf_by_treatment, index="treatment", columns="labels", values="count")
perf_by_treatment

In [18]:
import ast
def return_mean(x):
    x = ast.literal_eval(x)
    return np.mean(x)
df["avg_value"] = df["value"].apply(lambda x: return_mean(x))
df.head()

In [ ]:
unique_labels = df["labels"].unique()
palette = dict(zip(unique_labels, sns.color_palette("tab10", len(unique_labels))))

for treatment in df["treatment"].unique():
    temp = df[df["treatment"] == treatment]
    temp["index"] = temp["tst_id"] + "-" + temp["start_date"] + "-" + temp["end_date"]
    temp["has_incident"] = temp["has_incident"].replace({True: 1, False: 0})
    temp["predicted_incident"] = temp["predicted_incident"].replace({True: 1, False: 0})
    fig = plt.figure(figsize=(25, 6))
    ax = fig.add_subplot(111)
    ax.set_title(f"{treatment}")
    sns.scatterplot(data=temp, x="index", y="avg_value", hue="labels", palette=palette, ax=ax)
    sns.lineplot(data=temp, x="index", y="has_incident", ax=ax, color="gray", label="True Incident", linestyle=":")
    sns.lineplot(data=temp, x="index", y="predicted_incident", ax=ax, color="blue", label="Pred. Incident", linestyle="--")
    
    plt.xticks(rotation=90)
    plt.grid(True)
    plt.show()